In [ ]:

from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras import datasets, layers, models, regularizers, optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# 시간정보를 활용하여 폴더 생성
import datetime 

#3채널 525*700 픽셀의 500개 이미지
IMG_CHANNELS = 3
IMG_ROWS = 525
IMG_COLS = 700
IMG_SIZE = (IMG_ROWS, IMG_COLS)
#########################################################
window = 48
data = np.load('/content/drive/MyDrive/dataset/augmented_Spectogram/train_' + str(window)+'.npz')
X_train = data['array1']
Y_train = data['array2']

data = np.load('/content/drive/MyDrive/dataset/augmented_Spectogram/test_' + str(window)+'.npz')
argumented_X_test = data['array1']
argumented_Y_test = data['array2']

#print(X_train.shape)
val_size = int(X_train.shape[0]/5)
#print(val_size)
########################################################
k = 3 # k 는 1에서 5
argumented_X_val = X_train[val_size*(k-1):val_size*k, :, :, :]
argumented_Y_val = Y_train[val_size*(k-1):val_size*k]

argumented_X_train = np.concatenate((X_train[:val_size*(k-1), :, :, :] , X_train[val_size*k:, :, :, :] ),axis=0)
argumented_Y_train = np.concatenate((Y_train[:val_size*(k-1)] , Y_train[val_size*k:]),axis=0)

# 상수
BATCH_SIZE =  100
EPOCHS = 100
NUM_CLASSES = 5
VERBOSE = 1
OPTIM = tf.keras.optimizers.RMSprop()

INPUT_SHAPE = (428, 81, 3)

from keras.applications import inception_v3
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

base_model = tf.keras.applications.inception_v3.InceptionV3(
    include_top= False, weights= None, input_shape=INPUT_SHAPE, classes = NUM_CLASSES
    #classifier_activation='softmax'
)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation = "relu")(x)
predictions = Dense(5, activation = 'softmax')(x)

#model.add로 하는 방법..

model = Model(inputs = base_model.input, outputs = predictions)
#model.summary()
model.compile(loss = "categorical_crossentropy", optimizer = OPTIM, metrics = ["accuracy"])


# 텐서보드 사용
callbacks = [
    # 텐서보드 로그를 './logs' 디렉터리에 작성
    tf.keras.callbacks.TensorBoard(log_dir = './logs')
]

# 학습데이터의 log를 저장할 폴더 생성 (지정)
log_dir = "logs/my_board/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# 텐서보드 콜백 정의 하기
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(argumented_X_train, argumented_Y_train,
                    batch_size = BATCH_SIZE, epochs = EPOCHS,
                    verbose = VERBOSE, validation_data = (argumented_X_val, argumented_Y_val),
                    callbacks = [tensorboard_callback])

score = model.evaluate(argumented_X_test, argumented_Y_test, batch_size = BATCH_SIZE)
print("\nTest score:", score[0])
print("Test accuracy:", score[1])

 # 모델 저장
model_title = "/content/drive/MyDrive/model/coughdetection/inception_V3_" + str(window) +"_" + str(k) + ".json"
weights_title = '/content/drive/MyDrive/model/coughdetection/inception_V3_weights_' + str(window) +"_" + str(k) + ".h5"

model_json = model.to_json()
with open(model_title, 'w') as json_file:
  json_file.write(model_json)
model.save_weights(weights_title)



Mounted at /content/drive
Epoch 1/100
36/36 [==============================] - 1317s 36s/step - loss: 2.2252 - accuracy: 0.3792 - val_loss: 110.6458 - val_accuracy: 0.2000
Epoch 2/100
36/36 [==============================] - 1300s 36s/step - loss: 1.1080 - accuracy: 0.5650 - val_loss: 1506.0100 - val_accuracy: 0.2000
Epoch 3/100
28/36 [======================>.......] - ETA: 4:29 - loss: 0.8964 - accuracy: 0.6893